In [15]:
from src.preprocessing import remove_special_characters, remove_stopwords, preprocess, load_train_test_indices
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")
from sklearn.model_selection import train_test_split
import math
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from nltk.stem.porter import PorterStemmer

In [2]:
# read merged data
df = pd.read_csv("../data/final_merged_dataset.csv")
print("Example of merged data : ", df.sample(10))

Example of merged data :         Unnamed: 0                                              tweet  label
28809       29029  More tools to ensure the full 20 second hand w...      1
6695         6903  John and Edward Grimes have been praised on so...      1
29773       29993  Can someone please explain to me why Governor ...      1
31021       31242  Frequent hand washing is very important to pre...      1
6844         7052  Update from the @minhealthnz Today there are n...      1
27579       27799  WHO advice still stands: COVID-19 virus can be...      1
23414       23634  Depression, Anxiety, PTSD May Plague Many COVI...      1
18639       18857  #coronavirusFrance\n\nCDC says coronavirus RNA...      0
25542       25762  @RobertZayasNY @NYSPHSAA Summer camps and day ...      1
17411       17629  Vatican confirms Pope Francis and two aides te...      0


In [3]:
# preprocessing data
train, test, df = preprocess(df)
train_indices, test_indices = load_train_test_indices()
print("Example of processed data : ", train["preprocess"].head(5))

Example of processed data :  29533    coronavirus drug dexamethasone: covid-19 dexam...
22341    remember one? “if look (hydroxychloroquine) su...
5390     real sense, (oklahoma has) flattened curve. .....
22358    free horses! 52 thoroughbred horses need homes...
5612      58% coronavirus deaths scotland care facilities.
Name: preprocess, dtype: object


In [4]:
## vectorize texts and split train test dataset
corpus = df["preprocess"].tolist()
vect =CountVectorizer(min_df=0.,max_df=1.0)
X = vect.fit_transform(corpus)
X_train = X[train_indices, :]
X_test = X[test_indices, :]
y_train = train["label"]
y_test = test["label"]

In [19]:
# svclassifier = SVC(kernel='linear')
# svclassifier.fit(X_train, y_train)
# save model
# with open("../models/svm_v0.sav", "wb") as f:
#   pickle.dump(svclassifier, f)

In [17]:
# reload model
with open("../models/svm_v0.sav", "rb") as f:
  svclassifier = pickle.load(f)
# Predicting the Test set results
y_pred = svclassifier.predict(X_test)
accuracy_score(y_test,y_pred)

0.8959700093720713

In [14]:
# Examples of wrong prediction
n = 0
for ind, (prediction, label) in enumerate(zip(y_pred, y_test)):
  if prediction != label and n<10:
    n = n+1
    print("ORIGINAL TWEET IS : \n", test["tweet"].iloc[ind], "\n",
          "PROCESSED TWEET IS : \n", test["preprocess"].iloc[ind], "\n",
                                'has been classified as ', prediction,
                                'and should be ', label, "\n")


ORIGINAL TWEET IS : 
 I do really appreciate our Indian Government for this.. Giving a ray of hope to the Indians who are stuck in there.. But, they have to properly check their health status to confirm #Coronavirus negative..! https://t.co/Dijwh49VcJ 
 PROCESSED TWEET IS : 
 appreciate indian government this.. giving ray hope indians stuck there.. but, properly check health status confirm coronavirus negative..! 
 has been classified as  0 and should be  1 

ORIGINAL TWEET IS : 
 The COVID-19 pandemic is causing a reduction in child vaccinations... here's the concerns associated with that... https://t.co/Z0YxrF1wsj #Covid_19 #COVID #coronavirus #childvaccinations #vaccinations https://t.co/EqqxnDXwO0 
 PROCESSED TWEET IS : 
 covid-19 pandemic causing reduction child vaccinations... here's concerns associated that... covid19 covid coronavirus childvaccinations vaccinations 
 has been classified as  0 and should be  1 

ORIGINAL TWEET IS : 
 @annaamatorights @rolynhome @mi6rogue They al